#podcast summarizer

In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# !pip install langchain

In [ ]:
!pip install langchain --upgrade

In [ ]:
# !pip install openai

In [ ]:
!pip install openai --upgrade

In [ ]:
# from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.tools import BaseTool
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain_core.output_parsers import StrOutputParser

In [ ]:
llm  = ChatOpenAI(model_name = 'gpt-3.5-turbo',temperature=0)
llm.invoke('what 4+4?')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


AIMessage(content='4 + 4 = 8', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 13, 'total_tokens': 20}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-e8d299d1-fd99-4128-b2a2-e4a6580a4eb4-0')

In [ ]:
parser = StrOutputParser()
chain = llm | parser
chain.invoke('what is 4+4?')

'4 + 4 = 8'

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """ answer the question based on the context below, if you cant answer the question, reply "i dont know".
Context:{context}
question:{question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="x is m's sister",question="who is s's sister?")

'Human:  answer the question based on the context below, if you cant answer the question, reply "i dont know".\nContext:x is m\'s sister\nquestion:who is s\'s sister?\n'

In [ ]:
chain = prompt | llm | parser

chain.invoke({
    "context":"x is m's sister",
    "question":"who is x's sister?"
})

'm'

In [ ]:
translation_prompt = ChatPromptTemplate.from_template(
    "translate {answer} to {language}"

)

In [ ]:
from operator import itemgetter

translation_chain = (
    {"answer":chain, "language":itemgetter("language")} | translation_prompt | llm | parser
)

translation_chain.invoke(
    {
        "context":"x is m's sister",
        "question":"who is x's sister",
        "language":"spanish"
    }
)


'm translates to "metros" in Spanish.'

In [ ]:
!pip install whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=d9bfc0165d248c672310cbf0349c16a398f3e5e0535e221d61fefc44f18fd414
  Stored in directory: /root/.cache/pip/wheels/aa/7c/1d/015619716e2facae6631312503baf3c3220e6a9a3508cb14b6
Successfully built whisper


#getting the transcript of the YT video

In [ ]:
import tempfile
import whisper
from pytube import YouTube



if not os.path.exists("transcription1.txt"):

    youtube = YouTube('https://www.youtube.com/watch?v=e20l16WP1ZU&pp=ygUOMzAgbWluIHBvZGNhc3Q%3D')
    audio = youtube.streams.filter(only_audio=True).first()
    whisper_model = whisper.load_model("base")

    with tempfile.TemporaryDirectory() as tmpdir:
        file = audio.download(output_path=tmpdir)
        transcription = whisper_model.transcribe(file, fp16=False)["text"].strip()

        with open("transcription.txt", "w") as file:
            file.write(transcription)

In [ ]:
with open("transcription.txt","r") as f:
  data = f.read()
print(data)

Subscribers don't matter, views don't matter. Ask how can I make my videos better. Do that every single day for years. Everyone can have fun making videos, turning into a job, that's great. But you should be happy making videos, never making a dime off of it. The videos that I've made that are the best, I did it because I was so passionate, and I saw an opportunity to make something that felt like more than just like a YouTube video that I wanted to get views. Even a flop is going to do well long term. Your fans are your focus group. And if you don't listen to them, you're not going to be a successful creator long term. I waited 10 years to finally post something. I realized, oh my gosh, I hate doing this. Putting on makeup, taking it off, and then putting it on again. Oh my gosh, I hated it. It took me 10 years to figure out that the niche I thought I wanted to do was actually not what I wanted to do at all. I wasn't ready, I didn't know all the steps. I didn't have all the right answ

In [ ]:
try:
  chain.invoke({
      "context":data,
      "question":"how was lebron's career?"

  })
except Exception as e:
  print(e)

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("transcription.txt")
text_docs = loader.load()
text_docs

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
text_splitter.split_documents(text_docs)[:5]
documents = text_splitter.split_documents(text_docs)

In [ ]:
# from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
embedded_query = embeddings.embed_query("who is m's sister?")
print(embedded_query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


[-0.0023331434865317704, -0.010579751774412244, -0.005661903398788893, 0.002230392995472812, -0.03109440757440509, 0.027593806307595527, -0.003463398189688386, 0.003975378803790007, -0.008375931507393507, -0.03667836253354914, 0.008794019905079458, -0.00012312336149568754, 0.010282129951410542, 0.014023662504943456, -0.011685204658985385, -0.029393710112505973, 0.02137209096439252, -0.013003244959126557, 0.024079032173885732, -0.03939947847258773, 0.005413884747292855, 0.0008109312015828809, -0.0018601372086678911, -0.007886981569174119, 0.007872808702273887, 0.007631876949889251, -0.0027069423736464275, -0.026332456543732705, -0.038294024656692016, 0.007702739421745262, 0.015391305976590292, -0.0033199018704444783, -0.0017848459487362011, -0.017928179302538736, -0.004808011916775563, -0.008723157433223445, 0.009856955120274477, -0.004280087060241825, -0.01135923803350579, 0.006048103311610609, 0.009970335261508608, -0.007089780157777854, -0.006140224152494394, -0.003978922020515065, -

In [ ]:
sen1 = embeddings.embed_query("mary's sister is tom")
sen2 = embeddings.embed_query("hello this is a car")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

q1 = cosine_similarity([embedded_query],[sen1])
q2 = cosine_similarity([embedded_query],[sen2])


q1, q2

(array([[0.83888601]]), array([[0.71324129]]))

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vecstore1 = DocArrayInMemorySearch.from_texts(['hello this is tom',
                                    'merc is a good car',
                                    'iphones are overrated',
                                    'marys sister is yonda'],
                                   embedding=embeddings)


In [ ]:
vecstore1.similarity_search_with_score(query="who is marys sister?")

[(Document(page_content='marys sister is yonda'), 0.8764121700174807),
 (Document(page_content='hello this is tom'), 0.7220363365429516),
 (Document(page_content='merc is a good car'), 0.6973480388108557),
 (Document(page_content='iphones are overrated'), 0.6765176565295835)]

In [ ]:
retriever1 = vecstore1.as_retriever()
retriever1.invoke("what is a merc?")

[Document(page_content='merc is a good car'),
 Document(page_content='marys sister is yonda'),
 Document(page_content='iphones are overrated'),
 Document(page_content='hello this is tom')]

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
setup = RunnableParallel(context=retriever1,question=RunnablePassthrough())
setup.invoke("what phone is not good?")

{'context': [Document(page_content='iphones are overrated'),
  Document(page_content='merc is a good car'),
  Document(page_content='hello this is tom'),
  Document(page_content='marys sister is yonda')],
 'question': 'what phone is not good?'}

In [ ]:
chain = setup | prompt | llm |parser
chain.invoke("which car is good?")

'merc'

#now with this pipeline you can Q and A with the a YT video


In [ ]:
vectorstore2 = DocArrayInMemorySearch.from_documents(documents,embedding=embeddings)

chain = ({"context":vectorstore2.as_retriever(),"question":RunnablePassthrough()} | prompt | llm | parser)
chain.invoke("what is wors thing for my channels success?")

"The worst thing for your channel's success is when people try to copy or replicate a successful YouTube channel, as it is not likely to work."